In this notebook, we'll learn how to use pre-trained networks to solved challenging problem in computer vision. Specifically, you'll use networks tarined on ImageNet available from torchvision 

torchvision.models https://pytorch.org/docs/0.3.0/torchvision/models.html.

ImageNet is a massive dataset with over 1 million labeled iamges in 1000 categories. It is use dto train deep neural networks using an architecture called convolution layers. It's hard to explain convolutional networks in this notebook however, use this link to must know about this https://www.youtube.com/watch?v=2-Ol7ZB0MmU

Once trained, these models work astonishingly well as feature detectors for images they weren't trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we'll use transer learning to train a network that can classify our catand dog photos with near perfect accuracy.

With torchvision.models you can download these pre-trained networks and use them in your applications. We'll include in our imports now.

In [1]:
%matplotlib inline 
%config Inlinebackend.figure_format ='retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models


Most of the pre-trained models require the input 224x224 images.Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are [0.485, 0.456, 0.406] and the standard deviations are [0.229, 0.224, 0.225].

cat_dog dataset link click to download 
https://www.kaggle.com/c/dogs-vs-cats

In [2]:
data_dir = 'cat_dog_data'

# TODO : Define transforms for the training data and testing data.

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([ transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])
                                     ])

# Pass Transforms in here, the run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir +'/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir +'/test', transform=test_transforms)

trainloader =  torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader  =  torch.utils.data.DataLoader(test_data, batch_size=64)


We can load in a model such as 


DenseNet : https://pytorch.org/docs/0.3.0/torchvision/models.html#id5 


Let's print out the model architecture so we can see what's going on.

In [3]:
model = models.densenet121(pretrained=True)
model

D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional networks and overall works as feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer 

(classifier): Linear(in_features=1024, out_features=1000).

This layer was trained on the ImageNet dataset, so it wont work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. in general, I think about pre-trained networks qas amazingly good feature detectors can be used as the input for simple feed-forward classifiers.

In [6]:
# Freeze parameters so we don't backpropagation through them 
for param in model.parameters():
    param.requires_grad = False
    
from collections import OrderDict
classifier = nn.Sequential(OrderDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc1', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

ImportError: cannot import name 'OrderDict' from 'collections' (D:\Anaconda\Lib\collections\__init__.py)

With our model built, we need to train the classifier.However, now we're using a really deep neural network. If you try to train this on a CPU like normal, it will take a long , long time. Instaed, we're going to use the GPU to do the calculations.The linear algebra  computaions are done in parallel on the GPU leading to 100x increased training speeds.It's also possible to train on multiple GPU's, further decreasing training time.

PyTorch, along with pretty much every other deepl learning framwork, uses 
CUDA : https://developer.nvidia.com/cuda-zone to efficiently compute the forward and backward passes on the GPU.In PYToch, you move ypur model parameters and other tensors to the GPU memory using model.to('cuda'). You can move them back from the GPU with model.to('cpu') which you'll commonly do when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed. I'll compare how long it takes to perform a forward and backward pqass with and without a GPU.

In [7]:
model.cuda() 
# Model has all these parameters in there tensors that rae sitting in your memory on your computer,but we can move them over to GPU by saying model.cuda()

images.cuda()
# Similarly if you have tensor like your images, select images and if you want to run your images through model, you have to make sure that the tensors you're putting through your model or on the GPU if your model's on the GPU.so that to move a tensor from computer to the GPU.

model.cpu(), 




AssertionError: Torch not compiled with CUDA enabled

In [8]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.models import densenet121

# Define the model
model = densenet121(pretrained=True)

# Enable gradients for all model parameters
for param in model.parameters():
    param.requires_grad = True

# Define your data loaders as previously

for cuda in [False, True]:
    criterion = nn.CrossEntropyLoss()  # Change to CrossEntropyLoss for classification
    
    optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimize all model parameters
    
    if cuda:
        model.cuda()
    else:
        model.cpu()
        
    for ii, (inputs, labels) in enumerate(trainloader):
        if cuda:
            inputs, labels = inputs.cuda(), labels.cuda()
            
        start = time.time()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if ii == 3:
            break
            
    print(f"CUDA = {cuda}; Time per batch: {(time.time() - start)/3:.3f} seconds")


CUDA = False; Time per batch: 5.934 seconds


AssertionError: Torch not compiled with CUDA enabled

Instead of setting cuda manually, you can check if CUDA is available like cuda = torch.cuda.is_available().


In [13]:
cuda = torch.cuda.is_available()
cuda

False

In [14]:
# Exercise : Train a pre-trained models to classify the cat and dog iamges. Continue with the DenseNet model, or try ResNet, it's also a good model to try out first. MAke sure you're using only training the classifier and the pararmeters for the feature part are frozen.

In [18]:
# Use GPU if it's available

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [25]:
model = models.resnet50(pretrained=True)

# Turn off gradients for model / Frezze parameters so we don't bacprp through them
for param in model.parameters():
    param.requires_grad = False
    
# Define our new classifier
classifier = nn.Sequential(nn.Linear(2048, 512),
                           nn.ReLU(),
                           nn.Dropout(p=0.2),
                           nn.Linear(512, 2),
                           nn.Softmax(dim=1))
                           
model.fc = classifier

criterion = nn.NLLLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

model.to(device);

In [ ]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5

for epoch in range(epochs):
    for images, labels in trainloader:
        steps += 1
        
        # Move input and label tensors to the default device
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        logps = model(images)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if steps % print_every == 0:  
            model.eval()
            test_loss = 0
            accuracy = 0  
            for images, labels in testloader:  # Corrected the variable name from "intestloader" to "testloader"

                images, labels = images.to(device), labels.to(device)

                logps = model(images)
                loss = criterion(logps, labels)
                test_loss += loss.item()

                # Calculate accuracy
                ps = torch.exp(logps)
                top_p, top_class = ps.topk(1, dim=1)
                equality = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equality.type(torch.FloatTensor))

            print(f"Epoch {epoch + 1}/{epochs}.. "
                  f"Train loss: {running_loss / print_every:.3f}.. "
                  f"Test loss: {test_loss / len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy / len(testloader):.3f}")
            running_loss = 0
            model.train()  # Switch back to training mode after printing the statistics


In [ ]:
# Recommended: To wait it will take much more time load and to give you the output.